# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,56.52,77,65,30.15,SH,Sun Aug 4 21:30:48 2024
1,1,adamstown,-25.0660,-130.1015,69.40,81,100,16.91,PN,Sun Aug 4 21:30:50 2024
2,2,albany,42.6001,-73.9662,75.67,74,0,1.99,US,Sun Aug 4 21:26:17 2024
3,3,kirchberg in tirol,47.3907,12.3072,60.69,97,97,1.99,AT,Sun Aug 4 21:30:53 2024
4,4,geraldton,-28.7667,114.6000,52.21,87,0,9.22,AU,Sun Aug 4 21:17:18 2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
# Configure the map plot
every_city_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size= 'Humidity',
    scale = 1,
    color = 'City',
    alpha = 0.5,
    tiles = 'OSM',
    frame_width=700,
    frame_height=500
)

every_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[
    (city_data_df['Max Temp']>=55)&
    (city_data_df['Max Temp']<=60)&
    (city_data_df['Humidity']<=60)&
    (city_data_df['Cloudiness']<=50)&
    (city_data_df['Wind Speed']<=20)
]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,talnakh,69.4865,88.3972,56.62,50,0,11.74,RU,Sun Aug 4 21:31:19 2024
80,80,mount gambier,-37.8333,140.7667,56.82,50,9,3.76,AU,Sun Aug 4 21:32:54 2024
92,92,blackmans bay,-43.0167,147.3167,55.72,60,8,1.01,AU,Sun Aug 4 21:33:11 2024
270,270,kabompo,-13.5927,24.2008,55.22,33,0,4.05,ZM,Sun Aug 4 21:34:14 2024
348,348,amos,48.5669,-78.1162,56.34,58,18,5.30,CA,Sun Aug 4 21:39:33 2024
364,364,chipata,-13.6333,32.6500,57.24,46,5,4.68,ZM,Sun Aug 4 21:39:59 2024
433,433,qamdo,31.1667,97.2333,56.08,52,0,1.70,CN,Sun Aug 4 21:41:43 2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,talnakh,RU,69.4865,88.3972,50,
80,mount gambier,AU,-37.8333,140.7667,50,
92,blackmans bay,AU,-43.0167,147.3167,60,
270,kabompo,ZM,-13.5927,24.2008,33,
348,amos,CA,48.5669,-78.1162,58,
364,chipata,ZM,-13.6333,32.6500,46,
433,qamdo,CN,31.1667,97.2333,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel', 'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row.Lat
    lon = row.Lng

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
talnakh - nearest hotel: Талнах
mount gambier - nearest hotel: The Old Mount Gambier Gaol
blackmans bay - nearest hotel: Villa Howden
kabompo - nearest hotel: No hotel found
amos - nearest hotel: Hôtel des Eskers
chipata - nearest hotel: Kamocho Gueshouse
qamdo - nearest hotel: 银杏商务宾馆


,City,Country,Lat,Lng,Humidity,Hotel Name
20,talnakh,RU,69.4865,88.3972,50,Талнах
80,mount gambier,AU,-37.8333,140.7667,50,The Old Mount Gambier Gaol
92,blackmans bay,AU,-43.0167,147.3167,60,Villa Howden
270,kabompo,ZM,-13.5927,24.2008,33,No hotel found
348,amos,CA,48.5669,-78.1162,58,Hôtel des Eskers
364,chipata,ZM,-13.6333,32.6500,46,Kamocho Gueshouse
433,qamdo,CN,31.1667,97.2333,52,银杏商务宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size= 'Humidity',
    scale = 1,
    color = 'City',
    alpha = 0.5,
    tiles = 'OSM',
    hover_cols = ['Hotel Name', 'Country'],
    frame_width=700,
    frame_height=500
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)